# Исследование надёжности заёмщиков

Кредитный отдел банка заказал исследование - необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

## Загрузка файла с данными и изучение общей информации.

In [92]:
# импортируем необходимые библилтеки
import pandas as pd
from pymystem3 import Mystem 
from collections import Counter

In [93]:
data = pd.read_csv('/datasets/data.csv')
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


***После первого знакомства с датасетом отмечу следующее:***

- столбец с общим трудовым стажем имеет отрицательные значения;
- в столбце с образованием используется разный регистр написания текста;
- имеются пропуски в столбцах с общим трудовым стажем и ежемесячным доходом;
- в столбце "цель получения кредита" имеются схожие цели: к примеру жильё и недвижимость.


***Проверим общую информацию о таблице:***

In [94]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Посчитаем пропуски:**

In [95]:
# последовательно используем методы isna() и sum()
print('Количество пропусков в столбце с общим трудовым стажем -', data['days_employed'].isna().sum())
print('Количество пропусков в столбце с ежемесячным доходом -', data['total_income'].isna().sum())

Количество пропусков в столбце с общим трудовым стажем - 2174
Количество пропусков в столбце с ежемесячным доходом - 2174


**Узнаем, одинаковые ли это строки, где отсутствуют данные по столбцам 'days_employed' и 'total_income'**

In [96]:
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Предположение подтвердилось, пустые строки столбца 'days_employed' совпадают с пустыми строками столбца 'total_income', а значит пропуски не случайны. Скорее всего люди скрыли эти данные, и работник банка оставил ячейки пустыми.

***Необходимо проверить к одному типу занятости относятся эти пропуски или нет***

In [97]:
# применим метод value_counts, который возвращает уникальные значения и количество их упоминаний
data[(data['days_employed'].isna() == True) & (data['total_income'].isna() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Пропуски принадлежат разным типам занятости, а значит в последующей обработке данных придётся заполнять пропуски используя средние или медианные значения для конкретного типа занятости.

***Проверим числовые значения таблицы:***

In [98]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**После изучения общей информации о таблице выявлены следующие замечания:**
- Пропуски имеются только в двух столбцах и их количество совпадает;
- В столбце "дети" имеются подозрительные значения min = -1 и max = 20;
- В столбце "общий трудовой стаж" имеются некорректно введенные данные, т.к. среднее значение равно 63046 дней, что в реальности невозможно. Возможно стаж написан не в днях. Об этой неточности надо узнавать в банке. Но так как количество дней трудового стажа не понадобится для дальнейшего анализа, оставим значения без изменений.
- В столбце "возраст клиента" минимальное значение 0, что тоже говорит о том, что данные ввели неправильно.

**Вывод**

Статистика присланная из кредитного отдела банка представляет собой таблицу из 21525 строк и 12 столбцов. Для дальнейшей работы с таблицей и проведения анализа необходимо выполнить следующие действия:

- Значения столбца с общим трудовым стажем сделать положительными;
- Тип данных в столбце "общий трудовой стаж" и "ежемесячный доход" сделать целочисленными;
- Заполнить пропущенные значения в столбцах "общий трудовой стаж" и "ежемесячный доход";
- Некорректные значения в столбце "дети" заменить на корректные или же удалить строки с некорректными значениями, если их количество ничтожно мало;
- Проверить остальные столбцы на наличие некорректных данных и исправить их, если они будут.

## Предобработка данных

### Обработка пропусков

**Для начала проверим количество отрицательных значений в столбцах 'days_employed' и 'total_income':**

In [99]:
# посчитаем методом count()
print('Количество отрицательных значений в столбце "days_employed" - ', 
      data[data['days_employed'] < 0]['days_employed'].count())
print('Количество отрицательных значений в столбце "total_income" - ', 
      data[data['total_income'] < 0]['total_income'].count())

Количество отрицательных значений в столбце "days_employed" -  15906
Количество отрицательных значений в столбце "total_income" -  0


В столбце 'total_income' отрицательных значений нет. Количество отрицательных значений в столбце 'days_employed' составляет почти 74%, что указывает на ошибки при вводе данных в таблицу. Скорее всего при записи значения в таблицу использовали дефис. Поэтому можно смело привести данные столбца к положительным значениям.

In [100]:
# применим функцию abs(), которая вернет все значения по модулю
data['days_employed'] = abs(data['days_employed'])
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Заполним пропуски**

Для заполнения пропущенных значений в столбце 'days_employed' используем средние значения, а для значений в столбце 'total_income' - медианные значения, т.к. такой подход обеспечит наиболее правильные расчеты.

In [101]:
# используем метод apply,и вложим в него лямбда-функцию
# метод fillna() заполняет пропуски с типом NaN
# предварительно сгруппируем данные по типу занятости
data['days_employed'] = data.groupby('income_type')['days_employed'].apply(lambda x: x.fillna(x.mean()))
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

**Проверим информацию о таблице:**

In [102]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


От пропусков в данных избавились.

**Приведём значения столбца 'education' к единому регистру.**

In [103]:
# применим метод str.lower()
data['education'] = data['education'].str.lower()

**Проверим уникальные значения в столбце 'gender'**

In [104]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Выявлена одна строка с непонятным обозначением 'XNA'. Избавимся от нее и снова проверим столбец.

In [105]:
data = data.loc[data['gender'] != 'XNA']

In [106]:
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

**Найдем уникальные значения количества детей**

In [107]:
data['children'].value_counts()

 0     14148
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Из полученной информации видно, что значений -1 и 20 очень мало, что говорит о допущенных ошибках при вводе данных. Скорее всего работник, который вводил информацию, при записи использовал дефис, что привело к появлению -1, и случайно нажимал или задевал клавишу 0 при записи 2, что привело к появлению значения 20.

**Избавимся от данных ошибок путем замены значений -1 на 1 и 20 на 2.**

In [108]:
# для замены значений используем метод replace()
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)

In [109]:
data['children'].value_counts()

0    14148
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Проверим столбец 'dob_years' на наличие несовершеннолетних людей.**

In [110]:
print('Количество несовершеннолетних - ', data[data['dob_years'] < 18]['dob_years'].count())

Количество несовершеннолетних -  101


Количество несовершеннолетних ничтожно мало, поэтому, считаю, что от данных строк можно избавится. Их отсутствие не повлияет на результаты анализа.

In [111]:
data = data.loc[data['dob_years'] >= 18]

Снова проверим количество несовершеннолетних.

In [112]:
print('Количество несовершеннолетних - ', data[data['dob_years'] < 18]['dob_years'].count())

Количество несовершеннолетних -  0


**Вывод**

Пропуски в таблице заполнены, значения типа NaN отсутствуют. Изменены и устранены некорректные данные.

### Замена типа данных

Значения в столбцах с общим трудовым стажем и ежемесячным доходом приведём к целочисленным значениям.

In [113]:
# используем метод astype()
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
# проверим тип данных
data.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

**Вывод**

Значения трудового стажа и ежемесячного дохода привели к целочисленным значениям, так как данной точности достаточно для наших расчетов. 

### Обработка дубликатов

**Посчитаем количество дубликатов.**

In [114]:
# используем метод duplicated()
data.duplicated().sum()

71

Взглянем на эти дубликаты, чтобы выяснить причины их появления.

In [115]:
# применим метод sort_values, чтобы дубликаты увидеть упорядоченными
data[data.duplicated(keep=False)].sort_values(by=['days_employed', 'total_income'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2254,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
4081,1,2111,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости
5124,0,2111,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
8490,1,2111,31,среднее,1,женат / замужем,0,F,компаньон,0,172357,покупка жилья
9374,0,2111,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
20116,0,365003,57,среднее,1,гражданский брак,1,M,пенсионер,0,118514,свадьба
20187,0,365003,65,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
20702,0,365003,64,среднее,1,женат / замужем,0,F,пенсионер,0,118514,дополнительное образование
21032,0,365003,60,среднее,1,женат / замужем,0,F,пенсионер,0,118514,заняться образованием


Очень похоже на дублирование. Скорее всего, как и в предыдущих случаях, причина кроется в человеческом факторе - кто-то несколько раз вводил данные и не счел нужным себя проверить.

**Избавимся от данных дубликатов.**

In [116]:
# используем метод drop_duplicates()
data = data.drop_duplicates()
# сразу проверим результат
data.duplicated().sum()

0

**Вывод**

В таблице больше нет дублирующей информации. Итоговые результаты получатся более точными.

### Лемматизация

В последнем столбце нашей таблицы указана цели получения кредита. По факту, количества целей не так много, но из-за того, что при заполнении таблицы были использованы синонимы или просто схожие слова, то и количество целей кажется огромным.

**Для начала узнаем и посчитаем какие цели есть в нашем датасете.**

In [117]:
# выведем на экран количество уникальных целей
print(data['purpose'].value_counts())
# посчитаем общее количество уникальных целей
print('всего целей - ', len(data['purpose'].value_counts()))
# для менее затратной работы системы создадим список уникальных целей методом unique()
unique_purposes = data['purpose'].unique()

свадьба                                   786
на проведение свадьбы                     764
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     648
операции с жильем                         646
жилье                                     640
покупка жилья                             640
покупка жилья для семьи                   637
строительство собственной недвижимости    633
недвижимость                              629
операции со своей недвижимостью           627
строительство жилой недвижимости          621
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      617
ремонт жилью                              605
покупка жилой недвижимости                603
на покупку своего автомобиля              502
заняться высшим образованием      

**Лемматизируем список целей для получения уникальных лемм.**

In [118]:
# используем модуль pymystem3
m = Mystem()
# создадим список, в который будем добавлять леммы
list_of_lemmas = []
# циклом пройдемся по списку уникальных целей
for element in unique_purposes:
    lemma = m.lemmatize(element)
    list_of_lemmas.extend(lemma)
# используем контейнер для подсчета лемм
unique_lemmas = Counter(list_of_lemmas)
# выведем отсортированный контейнер
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[(' ', 59),
 ('\n', 38),
 ('покупка', 10),
 ('недвижимость', 10),
 ('автомобиль', 9),
 ('образование', 9),
 ('жилье', 7),
 ('с', 5),
 ('операция', 4),
 ('на', 4),
 ('свой', 4),
 ('свадьба', 3),
 ('строительство', 3),
 ('получение', 3),
 ('высокий', 3),
 ('дополнительный', 2),
 ('для', 2),
 ('коммерческий', 2),
 ('жилой', 2),
 ('заниматься', 2),
 ('сделка', 2),
 ('приобретение', 1),
 ('сыграть', 1),
 ('проведение', 1),
 ('семья', 1),
 ('собственный', 1),
 ('подержать', 1),
 ('со', 1),
 ('подержанный', 1),
 ('профильный', 1),
 ('сдача', 1),
 ('ремонт', 1)]

Для определения основных целей кредита будем использовать имена существительные. Как видно из сопоставления списка уникальных целей и отсортированного списка лемм, основными целями кредита являются: жильё, недвижимость, автомобиль, образование и свадьба. На основании этого создадим словарь, в котором ключ - лемма, а значение - измененная цель кредита.

In [119]:
# создаем словарь
dict_lem_purpose = dict({'свадьба':'свадьба', 
                    'жилье':'недвижимость', 
                    'недвижимость':'недвижимость',
                    'автомобиль':'автомобиль', 
                    'образование':'образование'})

Перед тем, как сделать столбец с измененными целями кредита, создадим функцию, с помощью которой и заполним новый столбец

In [120]:
# создаем функцию, которая будет принимать лемматизированную строку и возвращать значение из словаря
def purpose_category(row):
    str_lem_list = m.lemmatize(row)
    for lem in str_lem_list:
        if lem in dict_lem_purpose.keys():
            return dict_lem_purpose[lem]

Теперь создаем новый столбец.

In [121]:
data['purpose_change'] = data['purpose'].apply(purpose_category)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_change
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Проверим правильно ли заполнен новый столбец.

In [122]:
data['purpose_change'].value_counts()

недвижимость    10763
автомобиль       4284
образование      3995
свадьба          2310
Name: purpose_change, dtype: int64

**Вывод**

Лемматизация столбца с целями кредита выполнена. Создан столбец с обновленными целями кредита, теперь целей стало всего 4, вместо 38 из первоначальной таблицы.

### Категоризация данных

Категоризацию проводят с численными данными. Для данной работы в таблице есть три столбца: 'days_employed', 'dob_years' и 'total_income', имеющих целочисленные значения. Но так как для ответа на поставленные вопросы потребуется только столбец 'total_income', то категоризацию его данных и будем проводить.

Для начала узнаем минимальное и максимальное значение в столбце.

In [123]:
# используем методы min() и max()
print('минимальный доход - ', data['total_income'].min())
print('максимальный доход - ', data['total_income'].max())

минимальный доход -  20667
максимальный доход -  2265604


Федеральная служба государственной статистики разделяет уровень жизни в зависимости от доходов на следующие категории:
- крайняя нищета (доходы ниже прожиточного минимума — до 7-8 тыс.р.);
- нищета (доходы от одного до двух прожиточных минимума — от 8 до 12 тыс.р.);
- бедность (доходы от 12 до 20 тысяч рублей в месяц);
- выше бедности (доходы от 20 до 30 тысяч рублей в месяц);
- средний достаток — (доходы от 30 до 60 тысяч рублей в месяц);
- состоятельные — (доходы от 60 до 90 тысяч рублей в месяц);
- богатые — (доходы от 90 тысяч рублей в месяц);
- сверхбогатые — (доходы свыше 250 тысяч рублей в месяц);

Исходя из этой информации разделим доходы на 5 категорий:
* 1 категория бедный - до 30000;
* 2 категория средний - до 60000;
* 3 категория состоятельный - до 90000;
* 4 категория богатый - до 250000;
* 5 категория сверхбогатый - более 250000.

Добавим новые столбцы в таблицу 'income_rating' и 'income_rating_id' для дальнейшего анализа.

In [124]:
# создаем два пустых столбца
data['income_rating'] = 'NaN'
data['income_rating_id'] = 'NaN'
# с помощью условной индексации заполним новые столбцы
data.loc[data['total_income'] < 30000, ['income_rating', 'income_rating_id']] = ['бедный', 1]
data.loc[(30000 <= data['total_income']) & ( data['total_income'] < 60000), ['income_rating', 'income_rating_id']] = ['средний', 2]
data.loc[(60000 <= data['total_income']) & ( data['total_income'] < 90000), ['income_rating', 'income_rating_id']] = ['состоятельный', 3]
data.loc[(90000 <= data['total_income']) & ( data['total_income'] < 250000), ['income_rating', 'income_rating_id']] = ['богатый', 4]
data.loc[data['total_income'] >= 250000, ['income_rating', 'income_rating_id']] = ['сверхбогатый', 5]
# проверим количество уникальных значений в новых столбцах
print(data['income_rating'].value_counts())
data.head()

богатый          15221
сверхбогатый      2800
состоятельный     2529
средний            780
бедный              22
Name: income_rating, dtype: int64


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_change,income_rating,income_rating_id
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,сверхбогатый,5
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,богатый,4
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,богатый,4
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,сверхбогатый,5
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,богатый,4


**Вывод**

Разделил ежемесячные доходы на 5 категорий. В таблицу добавил два новых столбца для последующего анализа

## Ответы на вопросы проекта.

**- Есть ли зависимость между наличием детей и возвратом кредита в срок?**

Чтобы понять эту зависимость, создадим сперва функцию, которая будет принимать значения из рассматриваемого столбца и вовращать расчёты по задолженности.

In [125]:
# создаем функцию calculation_debt
# результат работы функии - таблица с расчетами по задолженности
def calculation_debt(column):
    final_table = data.groupby(column).agg({'debt':['sum', 'count']})
    final_table[('debt', 'proportion')] = final_table[('debt', 'sum')] / final_table[('debt', 'count')]
    return final_table

calculation_debt(['children'])

debt                  
           sum  count proportion
children                        
0         1058  14021   0.075458
1          442   4839   0.091341
2          202   2114   0.095553
3           27    328   0.082317
4            4     41   0.097561
5            0      9   0.000000

**Вывод**

Из полученной таблицы видно, что заемщики не имеющие детей немного более дисциплинированы. Доля невыплат среди людей, имеющих детей, примерно одинакова и колеблется в пределах 8 - 10 процентов, за исключением тех у кого 5 детей.  Можно сделать вывод, что у людей, не имющих детей меньше непредвиденных расходов, а многодетные участвуют в программе господдержки.

**- Есть ли зависимость между семейным положением и возвратом кредита в срок?**

Попробуем ответить на вопрос с помощью сводной таблицы

In [126]:
# выстраиваем сводную таблицу с помощью методов groupby() и agg()
data_pivot_family_status = data.groupby(['family_status_id', 'family_status']).agg({'debt':['sum', 'count']})
# добавим столбец с отношением
data_pivot_family_status[('debt', 'proportion')] = data_pivot_family_status[('debt', 'sum')] / data_pivot_family_status[('debt', 'count')]
data_pivot_family_status

debt                  
                                        sum  count proportion
family_status_id family_status                               
0                женат / замужем        927  12290   0.075427
1                гражданский брак       386   4129   0.093485
2                вдовец / вдова          62    954   0.064990
3                в разводе               85   1185   0.071730
4                Не женат / не замужем  273   2794   0.097709

**Вывод**

Наибольшее количество просрочек кредита у людей состоящих в офицальном браке, за ними следуют люди в гражданском браке. Такую особенность, вероятно, можно объяснить тем, что у состоящих в отношениях людей целей с денежными запросами больше, к примеру: ремонт, отпуск, праздничные торжества и т.п. Меньше всего просрочек у тех кто утратил своего супруга/супругу. Скорее всего за время совместной жизни они уже решили крупные финансовые вопросы, а сейчас берут кредиты на малые нужды, по которым просрочек не так много. Исходя из доли просрочек в 9,77 % делаем вывод, что все таки хуже дела обстоят с погашением кредита у не женатых/не замужних людей.

**- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

Для ответа на данный вопрос используем столбцы 'income_rating' и 'income_rating_id', которые были созданы при выполнении категоризации данных.

In [127]:
# используем ранее созданную функцию
calculation_debt(['income_rating_id', 'income_rating'])

debt                  
                                 sum  count proportion
income_rating_id income_rating                        
1                бедный            2     22   0.090909
2                средний          47    780   0.060256
3                состоятельный   213   2529   0.084223
4                богатый        1278  15221   0.083963
5                сверхбогатый    193   2800   0.068929

**Вывод**

Учитывая, что категории по доходу составлены по российским реалиям, то на основе полученной таблицы, я бы объединил 1 и 2 категории в бедных, 2 и 3 - в средний класс. По этим данным уже прослеживается общемировая тенденция - кредитуется больше всего средний класс, а значит и просрочек у него больше. Бедные практически не берут кредиты, так как опасаются не отдать. А богатым кредиты либо не нужны, либо они используют другие финансовые инструменты. И исходя из этой логики, доля просрочек больше у среднего класса - 8,4 %

**- Как разные цели кредита влияют на его возврат в срок?**

Для ответа на вопрос нам понадобится столбец с измененными целями кредита, полученный с помощью лемматизирования.


In [128]:
calculation_debt(['purpose_change'])

debt                  
                sum  count proportion
purpose_change                       
автомобиль      400   4284   0.093371
недвижимость    779  10763   0.072378
образование     370   3995   0.092616
свадьба         184   2310   0.079654

**Вывод**

Худшая по количеству не возврата цель кредита, которая занимает половину всех задолженностей, - это недвижимость. Скорее всего на это влияет ипотечное кредитование, особенно в иностранной валюте. Люди, потеряв работу или снизив доход, не думают о возврате кредита, а пытаются его отсрочить. Меньше всего количества просрочек по кредитам на свадьбу, я думаю из-за того, что, во-первых, объем кредита меньше, и во-вторых, добавляются новые родственники, которые помогают погасить долг. Но, судя по доле невозвратов, худшей целью является автомобиль - 9,33 % невозвратов.

## Общий вывод

Для общего вывода выведем еще одну сводную таблицу, учитывающую семейное положение и количество детей.

In [129]:
data_pivot_itog = data.groupby(['family_status', 'children']).agg({'debt':['sum', 'count']})
data_pivot_itog[('debt', 'proportion')] = data_pivot_itog[('debt', 'sum')] / data_pivot_itog[('debt', 'count')]
data_pivot_itog

debt                 
                                sum count proportion
family_status         children                      
Не женат / не замужем 0         210  2249   0.093375
                      1          51   452   0.112832
                      2          10    83   0.120482
                      3           1     8   0.125000
                      4           1     2   0.500000
в разводе             0          55   777   0.070785
                      1          21   315   0.066667
                      2           8    81   0.098765
                      3           1    11   0.090909
                      4           0     1   0.000000
вдовец / вдова        0          52   842   0.061758
                      1           7    81   0.086420
                      2           3    24   0.125000
                      3           0     6   0.000000
                      4           0     1   0.000000
гражданский брак      0         227  2716   0.083579
                      1         118   994   0.118712
                      2          33   353   0.093484
                      3           8    56   0.142857
                      4           0     8   0.000000
                      5           0     2   0.000000
женат / замужем       0         514  7437   0.069114
                      1         245  2997   0.081748
                      2         148  1573   0.094088
                      3          17   247   0.068826
                      4           3    29   0.103448
                      5           0     7   0.000000

Для выполнения заявленной цели проекта: определить влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, мной был выполнен ряд действий. В первую очередь были изучены данные, представленные кредитным отделом банка для работы. В ходе предобработки данных мной были заполнены пропущенные значения, исправлены некорректные данные и конкретезированы цели получения кредита. Удалено менее 0,5% строк датасета, что, как я считаю, не повлияло на итоговый результат. Применяя функции и группировку в сводные таблицы, нашел ответы на поставленные вопросы. 
**В итоге, отвечая на задание проекта, можно сказать следующее: люди состоящие в официальных отношениях и не имеющие детей, ответственнее других возвращают долг по кредиту. Заемщики, не состоящие в официальных отношениях или проживающие в гражданском браке и имеющие 2-3 детей, являются худшей группой по погашению кредита.**